# Практическая работа #1. Выполнил: Мешалкин Никита

Импорт необходимых для разработки библиотек:

In [4]:
pip install torchvision

  Using cached torchvision-0.16.0-cp38-cp38-win_amd64.whl (1.3 MB)
  Using cached torch-2.1.0-cp38-cp38-win_amd64.whl (192.3 MB)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: mpmath>=0.19 in c:\anaconda\lib\site-packages (from sympy->torch==2.1.0->torchvision) (1.1.0)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0



ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'C:\\ANACONDA\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [3]:
import os

import torch.nn
import torchvision.datasets
import torchvision.transforms
import matplotlib.pyplot as plt
import time

ModuleNotFoundError: No module named 'torchvision'

Загрузка датасета (CIFAR10):

In [2]:
dir_name = os.getcwd()
train_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = True, download = True,
    transform = torchvision.transforms.ToTensor()
)
test_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = False, download = True,
    transform = torchvision.transforms.ToTensor()
)

NameError: name 'torchvision' is not defined

In [ ]:
train_dataset

Покажем что классы сбаллансированы

In [ ]:
counts = [0] * 10
for i in range(len(train_dataset)):
  counts[train_dataset[i][1]] += 1
print(counts)

In [ ]:
fig, axes = plt.subplots(2, 5, figsize = (15, 5))
for label in range(10):
  for i in range(len(train_dataset)):
    if label == train_dataset[i][1]:
      axes[label // 5, label % 5].imshow(train_dataset[i][0].permute(1, 2, 0))
      break
plt.show()

Создадим модель нейронной сети

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.model = torch.nn.Sequential(
                # 100 x ((32-3+2*1)/1+1) x ((32-3+2*1)/1+1) = 100 x 32 x 32
                torch.nn.Conv2d(3, 100, kernel_size=3, padding=1),
                torch.nn.ReLU(),
                # 300 x ((32-3+2*1)/1+1) x ((32-3+2*1)/1+1) = 300 x 32 x 32
                torch.nn.Conv2d(100, 300, kernel_size=3, padding=1),
                torch.nn.ReLU(),
                # 300 x ((32-3)/2+1) x ((32-3)/2+1) = 300 x 15 x 15
                torch.nn.MaxPool2d(kernel_size = 3, stride = 2),
                torch.nn.ReLU(),
                torch.nn.Flatten(),
                torch.nn.Linear(300*15*15, 10))

    # Переопределение метода, вызываемого в процессе прямого прохода
    def forward(self, x):
        return self.model(x)

model = Model()
model

In [ ]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = 50, shuffle = True
)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = 50, shuffle = False
)

Обучим модель

In [ ]:
model = Model()

learning_rate = 0.1
num_epochs = 20

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

def get_accuracy(data_loader, model):
    tp = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n += labels.size(0)
            tp += (predicted == labels).sum()
    return tp / n

total_time = 0

for epoch in range(num_epochs):
    start = time.time()
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    epoch_time = time.time() - start
    total_time += epoch_time

    print(f'Epoch {epoch}')
    print(f'    accuracy on train data : {get_accuracy(train_data_loader, model)}')
    print(f

Покажем результирующую точность на тестовой выборке и общее время обучения модели



In [ ]:
print(f'accuracy on test data  : {get_accuracy(test_data_loader, model)}\n')
print(f'total time = {total_time}')